In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def writeNewtxt(file_path, df):
    """Writes a pandas dataframe into txt, columns separated by tabs ("\t")
    """
    f = open(file_path, 'w')
    str_cols = ""
    for col in df.columns[:-1]:
        str_cols += col + "\t"
    str_cols += df.columns.values[-1]+ "\n"
    f.write(str(str_cols))
    for row in df.values:
        str_row = ""
        for col in row[:-1]:
            str_row += str(col) + "\t"
        str_row += str(row[-1])+ "\n"
        f.write(str_row)
    f.close

In [15]:
gvs_chr20_read= "/home/androx/Documents/trabalho/datasets/Genomic_variants/chr20/allChr20.GRCH38.sorted.vcf"
gvs_chr20_df = pd.read_csv(gvs_chr20_read, sep="\t", usecols=['#CHROM',  'POS', 'REF', 'ALT', 'INFO'])

gvs_chr20_df

,#CHROM,POS,REF,ALT,INFO
0,20,60114,T,C,AC=1;AN=5096;DP=67920;AF=0;EAS_AF=0;EUR_AF=0;A...
1,20,60138,T,A,AC=1;AN=5096;DP=69096;AF=0;EAS_AF=0;EUR_AF=0;A...
2,20,60149,C,T,AC=3;AN=5096;DP=66846;AF=0;EAS_AF=0;EUR_AF=0;A...
3,20,60424,A,C,AC=0;AN=5096;DP=29861;AF=0;EAS_AF=0;EUR_AF=0;A...
4,20,60440,A,T,AC=2;AN=5096;DP=26998;AF=0;EAS_AF=0;EUR_AF=0;A...
...,...,...,...,...,...
1817487,20,64333367,G,A,AC=6;AN=5096;DP=16236;AF=0;EAS_AF=0;EUR_AF=0;A...
1817488,20,64333468,C,T,AC=1;AN=5096;DP=13363;AF=0;EAS_AF=0;EUR_AF=0;A...
1817489,20,64333615,G,A,AC=16;AN=5096;DP=13225;AF=0;EAS_AF=0;EUR_AF=0;...
1817490,20,64333621,G,A,AC=10;AN=5096;DP=13129;AF=0;EAS_AF=0;EUR_AF=0;...


In [4]:
#writeNewtxt(gvs_chr20_read, gvs_chr20_df)

In [5]:
#gvs_chr20_df["INFO"][0]
#gvs_chr20_df["INFO"][30]

### Dividir as genomic variants em SNPs e INDELs

In [6]:
#snps_chr20_df = get_df_snp(gvs_chr20_df)
#snps_chr20_df
#mask1 = (gvs_chr20_df['REF'].str.len() == 1) & (gvs_chr20_df['ALT'].str.len() == 1)
#snps_chr20_df = gvs_chr20_df.loc[mask1]
#snps_chr20_df

In [7]:
#indels_chr20_df = get_df_indels(gvs_chr20_df)
#indels_chr20_df
#mask2 = (gvs_chr20_df['REF'].str.len() > 1) | (gvs_chr20_df['ALT'].str.len() > 1)
#indels_chr20_df = gvs_chr20_df.loc[mask2]
#indels_chr20_df

## Seperated density values for insertions and deletions

In [8]:
#inserts_chr20 = indels_chr20_df.loc[(indels_chr20_df['REF']) < (indels_chr20_df['ALT'])]
#inserts_chr20

In [9]:
#deletions_chr20 = indels_chr20_df.loc[(indels_chr20_df['REF']) > (indels_chr20_df['ALT'])]
#deletions_chr20

## Funções de calculo de densidades

In [5]:
def calc_snpDensity(cb_df_chr, gv_df_chr):#, chrom = "chr20"):
    """calculates the density of snps within the cytobands of one chromosome
    """
    densities = []
    #chr_cytobands = cytobands_df.loc[cytobands_df['Chromossome'] == chrom]
    chr_size = cb_df_chr["Last_index"][cb_df_chr.index[-1]]+1
    chr_gvs = np.zeros((chr_size), dtype=int)
    #percorrer o dataset pelas vulnerabilidades no cromossoma e anotar na lista np.zeroes
    for row in gv_df_chr.index:
        chr_gvs[gv_df_chr["POS"][row]]+=1
    #Calculo da densidade para cada cytoband
    for cytoband in cb_df_chr.index:
        cytoband_array = chr_gvs[int(cb_df_chr["First_index"][cytoband]):int(cb_df_chr["Last_index"][cytoband])+1]
        snp_indel_length = np.count_nonzero(cytoband_array)
        cytoband_length = cb_df_chr["Size"][cytoband]
        density = snp_indel_length/cytoband_length
        densities.append(round(density, 3))
        
    cb_df_chr.insert(len(cb_df_chr.columns), "SNP_density", densities, True)
    return cb_df_chr

In [6]:
def calc_indelDens(cb_df_chr, gv_df_chr):#, chrom = "chr20"):
    """calculates the density of indels, inserts or deletes within the cytobands of one chromosome
    """
    densities = []
    #chr_cytobands = cytobands_df.loc[cytobands_df['Chromossome'] == chrom]
    chr_size = cb_df_chr["Last_index"][cb_df_chr.index[-1]]+1
    chr_gvs = np.zeros((chr_size), dtype=int)
    #percorrer o dataset pelas vulnerabilidades no cromossoma e anotar na lista np.zeroes
    for row in gv_df_chr.index:
        if len(gv_df_chr["REF"][row]) > len(gv_df_chr["ALT"][row]):               #Deletion
            chr_gvs[gv_df_chr["POS"][row]:gv_df_chr["POS"][row] + len(gv_df_chr["REF"][row])+1]+=1
        else:                                                                    #Insertion
            chr_gvs[gv_df_chr["POS"][row]:gv_df_chr["POS"][row] + len(gv_df_chr["ALT"][row])+1]+=1
    #Calculo da densidade para cada cytoband
    for cytoband in cb_df_chr.index:
        cytoband_array = chr_gvs[int(cb_df_chr["First_index"][cytoband]):int(cb_df_chr["Last_index"][cytoband])+1]
        snp_indel_length = np.count_nonzero(cytoband_array)
        cytoband_length = cb_df_chr["Size"][cytoband]
        density = snp_indel_length/cytoband_length
        densities.append(round(density, 3))
        
    cb_df_chr.insert(len(cb_df_chr.columns), "Indel_density", densities, True)
    return cb_df_chr

In [12]:
def calc_insert_deleteDens(cb_df_chr, gv_df_chr, dens_type = "Insertion_density"):#, chrom = "chr20"):
    """calculates the density of inserts or deletes within the cytobands of one chromosome
    """
    densities = []
    #chr_cytobands = cytobands_df.loc[cytobands_df['Chromossome'] == chrom]
    chr_size = cb_df_chr["Last_index"][cb_df_chr.index[-1]]+1
    chr_gvs = np.zeros((chr_size), dtype=int)
    #percorrer o dataset pelas vulnerabilidades no cromossoma e anotar na lista np.zeroes
    for row in gv_df_chr.index:
        if dens_type == "Inserts_density":
            chr_gvs[gv_df_chr["POS"][row]:gv_df_chr["POS"][row] + len(gv_df_chr["ALT"][row])+1]+=1
        elif dens_type == "Deletes_density":
            chr_gvs[gv_df_chr["POS"][row]:gv_df_chr["POS"][row] + len(gv_df_chr["REF"][row])+1]+=1
    #Calculo da densidade para cada cytoband
    for cytoband in cb_df_chr.index:
        cytoband_array = chr_gvs[int(cb_df_chr["First_index"][cytoband]):int(cb_df_chr["Last_index"][cytoband])+1]
        snp_indel_length = np.count_nonzero(cytoband_array)
        cytoband_length = cb_df_chr["Size"][cytoband]
        density = snp_indel_length/cytoband_length
        densities.append(round(density, 3))
        
    cb_df_chr.insert(len(cb_df_chr.columns), dens_type, densities, True)
    return cb_df_chr

In [13]:
cytoPath_read = "/home/androx/Documents/trabalho/citobands/cytobandFiltered_processed_str_dr_dg.txt"
cytoPath_write = "/home/androx/Documents/trabalho/citobands/cytobandFiltered_processed_str_dr_dg_gv.txt"

cytobands_df = pd.read_csv(cytoPath_read, sep="\t")#, header=None))

cytobands_df

,Chromossome,First_index,Last_index,Citoband,Unkown,Size,STR_Density,DR_density,CDSs_density,CDSIntron_density,CDSExon_density,5P_UTR_density,UTR5Intron_density,UTR5Exon_density,3P_UTR_density,UTR3Intron_density,UTR3Exon_density,Exons_density,Introns_density
0,chr1,0,2299999,p36.33,gneg,2299999,0.088,0.326,0.038,0.022,0.015,0.033,0.022,0.021,0.012,0.025,0.019,0.171,0.069
1,chr1,2300000,5299999,p36.32,gpos25,2999999,0.084,0.282,0.016,0.011,0.005,0.022,0.010,0.014,0.004,0.013,0.006,0.079,0.034
2,chr1,5300000,7099999,p36.31,gneg,1799999,0.056,0.275,0.024,0.010,0.008,0.034,0.009,0.020,0.006,0.012,0.010,0.082,0.031
3,chr1,7100000,9099999,p36.23,gpos25,1999999,0.049,0.405,0.013,0.004,0.002,0.018,0.003,0.010,0.005,0.004,0.004,0.047,0.011
4,chr1,9100000,12499999,p36.22,gneg,3399999,0.056,0.404,0.030,0.010,0.008,0.036,0.010,0.019,0.005,0.014,0.010,0.093,0.034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,chrY,12400000,17099999,q11.221,gpos50,4699999,0.037,0.015,0.004,0.001,0.001,0.003,0.001,0.003,0.001,0.001,0.003,0.016,0.003
859,chrY,17100000,19599999,q11.222,gneg,2499999,0.062,0.013,0.002,0.004,0.004,0.001,0.004,0.003,0.000,0.004,0.002,0.045,0.012
860,chrY,19600000,23799999,q11.223,gpos50,4199999,0.082,0.006,0.005,0.004,0.002,0.004,0.004,0.002,0.001,0.005,0.002,0.036,0.013
861,chrY,23800000,26599999,q11.23,gneg,2799999,0.026,0.002,0.003,0.002,0.000,0.002,0.003,0.001,0.001,0.003,0.001,0.034,0.008


### Cytoband data frame only chr20

In [14]:
#chr20_df = cytobands_df.loc[cytobands_df['Chromossome'] == "chr20"]

### Calculate the different densities

In [15]:
#chr20_df = calc_snpDensity(chr20_df, snps_chr20_df)

In [16]:
#chr20_df = calc_indelDens(chr20_df, indels_chr20_df)

In [17]:
#chr20_df = calc_insert_deleteDens(chr20_df, inserts_chr20, "Insertion_density")

In [18]:
#chr20_df = calc_insert_deleteDens(chr20_df, deletions_chr20, "Deletion_density")

In [19]:
#chr20_df

### Make plots of the data

In [20]:
def plot_1chr(chr_df, chromossome, col = "SNP_density"):
    plt.figure(figsize=(30,10))
    plt.plot(chr_df["Citoband"], chr_df[col], 'ro--', linewidth=2, markersize=6)
    plt.xlabel("Cytoband")
    plt.ylabel(col)
    plt.grid()
    plt.title(col + " in chromosome "  + chromossome)
    #plt.show()
    plt.savefig(chromossome + col)

In [21]:
#plot_1chr(chr20_df, "20", "SNP_density")

In [22]:
#plot_1chr(chr20_df, "20", "Indel_density")

In [23]:
#cytoPath_write = "/home/androx/Documents/trabalho/citobands/Latest_gvs/cytobandFiltered_processed_str_dr_dg_gv_chr20.txt"
#writeNewtxt(cytoPath_write, chr20_df)

# Codigo de execução para cada um dos chrs

In [24]:
def execute_1chr(chromosome):
    """
    """
    gvs_chr_read= "/home/androx/Documents/trabalho/datasets/Genomic_variants/" + chromosome + "/" + chromosome + "_gvs.vcf"
    cytoband_write = "/home/androx/Documents/trabalho/citobands/Latest_gvs/cytobandFiltered_processed_str_dr_dg_gv_" + chromosome + ".txt"
    
    print(gvs_chr_read)
    gvsChr_df = pd.read_csv(gvs_chr_read, sep="\t")           #genomic variants dataset

    ####### Separation of gv dataset into snps, indels, inserts and delets datasets ############
    mask1 = (gvsChr_df['REF'].str.len() == 1) & (gvsChr_df['ALT'].str.len() == 1)
    mask2 = (gvsChr_df['REF'].str.len() > 1) | (gvsChr_df['ALT'].str.len() > 1)
    
    snps_chr_df = gvsChr_df.loc[mask1]
    indels_chr_df = gvsChr_df.loc[mask2]
    inserts_chr_df = indels_chr_df.loc[(indels_chr_df['REF']) < (indels_chr_df['ALT'])]
    deletes_chr_df = indels_chr_df.loc[(indels_chr_df['REF']) > (indels_chr_df['ALT'])]

    #print(indels_chr_df)
    ####### Calculation of densities into the cytoband dataset
    chr_df = cytobands_df.loc[cytobands_df['Chromossome'] == chromosome]    #cytobands dataset of intended chromosome
    
    chr_df = calc_snpDensity(chr_df, snps_chr_df)
    chr_df = calc_indelDens(chr_df, indels_chr_df)
    chr_df = calc_insert_deleteDens(chr_df, inserts_chr_df, "Inserts_density")
    chr_df = calc_insert_deleteDens(chr_df, deletes_chr_df, "Deletes_density")
    print(chr_df)

    writeNewtxt(cytoband_write, chr_df)

In [32]:
execute_1chr("chrX")

/home/androx/Documents/trabalho/datasets/Genomic_variants/chrX/chrX_gvs.vcf
    Chromossome  First_index  Last_index Citoband   Unkown     Size  \
812        chrX            0     4399999   p22.33     gneg  4399999   
813        chrX      4400000     6099999   p22.32   gpos50  1699999   
814        chrX      6100000     9599999   p22.31     gneg  3499999   
815        chrX      9600000    17399999    p22.2   gpos50  7799999   
816        chrX     17400000    19199999   p22.13     gneg  1799999   
817        chrX     19200000    21899999   p22.12   gpos50  2699999   
818        chrX     21900000    24899999   p22.11     gneg  2999999   
819        chrX     24900000    29299999    p21.3  gpos100  4399999   
820        chrX     29300000    31499999    p21.2     gneg  2199999   
821        chrX     31500000    37799999    p21.1  gpos100  6299999   
822        chrX     37800000    42499999    p11.4     gneg  4699999   
823        chrX     42500000    47599999    p11.3   gpos75  5099999   
8

### Juntar todos os dataframes criados cromossoma a cromossoma

In [10]:
def concat_dfs():
    frames = []
    chroms = ["chr1", "chr2", "chr3", "chr4", "chr5", "chr6", "chr7", "chr8", "chr9", "chr10", "chr11", "chr12", "chr13", "chr14", "chr15", "chr16", "chr17", "chr18", "chr19", "chr20", "chr21", "chr22", "chrX"]
    for chromosome in chroms:
        gvs_chr_read= "/home/androx/Documents/trabalho/citobands/Latest_gvs/cytobandFiltered_processed_str_dr_dg_gv_" + chromosome + ".txt"
        chr_df = pd.read_csv(gvs_chr_read, sep="\t")
        frames.append(chr_df)
    result = pd.concat(frames)
    return result

In [12]:
cb_df_sorted = concat_dfs()

In [13]:
cb_df_sorted

,Chromossome,First_index,Last_index,Citoband,Unkown,Size,STR_Density,DR_density,CDSs_density,CDSIntron_density,...,UTR5Exon_density,3P_UTR_density,UTR3Intron_density,UTR3Exon_density,Exons_density,Introns_density,SNP_density,Indel_density,Inserts_density,Deletes_density
0,chr1,0,2299999,p36.33,gneg,2299999,0.088,0.326,0.038,0.022,...,0.021,0.012,0.025,0.019,0.171,0.069,0.022,0.006,0.002,0.005
1,chr1,2300000,5299999,p36.32,gpos25,2999999,0.084,0.282,0.016,0.011,...,0.014,0.004,0.013,0.006,0.079,0.034,0.033,0.009,0.002,0.007
2,chr1,5300000,7099999,p36.31,gneg,1799999,0.056,0.275,0.024,0.010,...,0.020,0.006,0.012,0.010,0.082,0.031,0.031,0.009,0.002,0.007
3,chr1,7100000,9099999,p36.23,gpos25,1999999,0.049,0.405,0.013,0.004,...,0.010,0.005,0.004,0.004,0.047,0.011,0.028,0.010,0.002,0.007
4,chr1,9100000,12499999,p36.22,gneg,3399999,0.056,0.404,0.030,0.010,...,0.019,0.005,0.014,0.010,0.093,0.034,0.028,0.009,0.002,0.006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,chrX,134500000,138899999,q26.3,gneg,4399999,0.028,0.144,0.011,0.005,...,0.012,0.004,0.007,0.013,0.053,0.017,0.000,0.000,0.000,0.000
36,chrX,138900000,141199999,q27.1,gpos75,2299999,0.018,0.078,0.005,0.002,...,0.004,0.001,0.002,0.002,0.027,0.006,0.000,0.000,0.000,0.000
37,chrX,141200000,142999999,q27.2,gneg,1799999,0.023,0.014,0.004,0.003,...,0.001,0.001,0.003,0.001,0.015,0.009,0.000,0.000,0.000,0.000
38,chrX,143000000,147999999,q27.3,gpos100,4999999,0.020,0.018,0.002,0.002,...,0.002,0.001,0.002,0.002,0.014,0.005,0.000,0.000,0.000,0.000


In [14]:
file_write = "/home/androx/Documents/trabalho/citobands/Latest_gvs/cytobandFiltered_processed_str_dr_dg_gv_ALLCHR.txt"
writeNewtxt(file_write, cb_df_sorted)